# NELL Test Tool

## Libs

### Setup

In [168]:
from IPython.display import clear_output

%pip install selenium 
clear_output(wait=True)

%pip install webdriver_manager 
clear_output(wait=True)

%pip install beautifulsoup4
clear_output(wait=True)

%pip install pandas 
clear_output(wait=True)

%pip install ipywidgets
clear_output()

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.by import By
#from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
from collections import defaultdict
from xml.sax.saxutils import quoteattr
from IPython.display import display #, Javascript
from ipywidgets import HTML

import ipywidgets as widgets
import pandas as pd
import time

print("All packages installed")

All packages installed


### Driver Bingings

In [170]:
prefs = {"profile.default_content_setting_values.notifications": 1}
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("prefs", prefs)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.quit()


def new_window(url, wait=0):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(wait)
    return driver


def highlight_element(driver, selector):
    script = f"""
    var element = document.evaluate('{selector}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) {{
        var originalStyle = element.style.border;
        var count = 0;
        var interval = setInterval(function() {{
            count += 1;
            element.style.border = count % 2 ? '3px solid red' : '';
            if (count > 9) {{
                clearInterval(interval);
                element.style.border = originalStyle;
            }}
        }}, 500);
    }}
    """
    try:
        driver.execute_script(script)
    except Exception as e:
        print("Error while highlighting the element:", e)


# generate keys
def generate_key(tag, element, counter):
    base = {"input": "txt", "button": "btn", "a": "lnk", "img": "img"}
    prefix = base.get(tag, "elem")

    parts = [prefix]
    
    txt = element.get_text(strip=True)
    if txt:
        parts.append(txt.replace(" ", "_"))
        return key_builder(parts, counter)
    
    if element.get('name'):
        parts.append(element['name'])
        return key_builder(parts, counter)
    
    if element.get('aria-label'):
        parts.append(element['aria-label'].replace(" ", "_"))
        return key_builder(parts, counter)

    if element.get('id'):
        parts.append(element['id'])
        return key_builder(parts, counter)
    
    if element.get('data-test-id'):
        parts.append(element['data-test-id'])
        return key_builder(parts, counter)
    
    return key_builder(parts, counter)


def key_builder(parts, counter):
    key = '_'.join(filter(None, parts)).lower().replace("-", "_")
    counter[key] += 1
    return key if counter[key] == 1 else f"{key}_{counter[key]}"


# Generate Selectors
def generate_selector(element, driver):
    selectors = []

    for attr in ['id', 'name', 'placeholder', 'aria-label', 'data-test-id', 'alt']:
        if element.get(attr):
            value = quoteattr(element[attr])
            selectors.append(f"@{attr}={value}")


    if selectors:
        xpath_selector = f"//{element.name}[{' and '.join(selectors)}]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    txt = element.get_text(strip=True)
    if txt:
        xpath_selector = f"//{element.name}[contains(text(), {quoteattr(txt)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    if element.get('class'):
        classes = '.'.join(element.get('class'))
        xpath_selector = f"//{element.name}[contains(@class, {quoteattr(classes)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector

    return None


# read page objects
def read_page_objects_metadata(driver):

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    elements = soup.find_all(['input', 'button', 'a', 'img'])

    counter = defaultdict(int)
    result = defaultdict(list)

    for element in elements:
        tag = element.name
        selector = generate_selector(element, driver)
        key = generate_key(tag, element, counter)

        result[tag].append({
            "key": key,
            "selector": selector,
            "attributes": {attr: element.get(attr) for attr in element.attrs},
            "text": element.get_text(strip=True),
            "visibility": "visible" if element.get('type') != 'hidden' else "invisible"
        })
    return result

print("Done!")

Done!


## GUI

### Widgets

In [174]:
class Toolbar:
    def __init__(self):
        self.new_window_button = widgets.Button(description="New Window") 
        self.inspect_button = widgets.Button(description="Inspect") 
        self.filter_text = widgets.Text(placeholder='Type to filter...') 
        self.content = widgets.HBox([self.new_window_button, 
                                   self.inspect_button, 
                                   self.filter_text])

class Table:
    def __init__(self, df=None):
        self.content = widgets.HBox()

        self.data_event = None
        self.web_event = None

        self.data = df if df is not None else pd.DataFrame(
            columns=["Alias", "Text", "Web", "Export", "Data", "Type", "Att"])
        self.reload()

    def try_fire_data_event(self, b, index):
        if self.data_event is not None:
            dt = self.data["Data"][index]
            self.data_event(dt)

    def try_fire_export_event(self, b, index):
        dt = self.data["Data"][index]
        self.data_event(dt)

        if b.description == "N":
            b.description = "Y"
            return
        b.description = "N"

    def try_fire_web_event(self, b, index):
        dt = self.data["Data"][index]
        self.data_event(dt)
        
        global test_window
        highlight_element(test_window, self.data["Web"][index])

    def reload(self, df=None):

        if df is not None: self.data = df
        df = self.data
        lines = len(self.data) + 1
        grid = widgets.GridspecLayout(lines, 2)

        for i in range(lines):
            if i == 0: 
                grid[i, 0] = widgets.HTML(value="<b>Alias</b>")
                continue
            
            cell_value_alias = getattr(df.iloc[i-1], "Alias", '') if i > 0 else ''
            cell_value_web = getattr(df.iloc[i-1], "Web", '') if i > 0 else ''

            pnl = widgets.HBox([
                widgets.Button(description="N", layout=widgets.Layout(width='50px', height='30px')),
                widgets.Button(tooltip=cell_value_web, icon='search', layout=widgets.Layout(width='50px', height='30px'))
            ], layout=widgets.Layout(width='auto', height='auto', weight='0'))

            pnl.children[0].on_click(lambda b, index=i-1: self.try_fire_export_event(b, index))
            pnl.children[1].on_click(lambda b, index=i-1: self.try_fire_web_event(b, index))

            grid[i, 0] = widgets.Text(value=str(cell_value_alias), layout=widgets.Layout(width='250px', weight='2'))        
            grid[i, 1] = pnl

        self.content.children = [grid]


class Properties:
    
    def __init__(self, properties={}):        
        self.content = widgets.HBox()
        self.reload(properties)

    def reload(self, properties={}):
        num_props = len(properties)
        grid = widgets.GridspecLayout(num_props + 1, 2, width='auto')
        grid[0, 0] = widgets.HTML(value="<b>Property</b>", layout=widgets.Layout(width='80px'))
        grid[0, 1] = widgets.HTML(value="<b>Value</b>", layout=widgets.Layout(width='600px'))  

        i = 0
        for i, (key, value) in enumerate(properties.items(), start=1):
            grid[i, 0] = widgets.Text(value=key, layout=widgets.Layout(width='80px'), disabled=True, bold=True)
            grid[i, 1] = widgets.Text(value=str(value), layout=widgets.Layout(width='400px'))

        self.content.children = [grid]


def new_cell(content, width='100%', height='100px', scroll=False):
    return widgets.Box([content], layout=widgets.Layout(
        border='1px solid black',
        width=width,
        height=height,
        overflow='auto' if scroll else 'hidden'
    ))


class Window():

    def __init__(self, new_window_event, inspect_event):
         
        self.new_window_event = new_window_event
        self.inspect_event = inspect_event

        self.dataframe = pd.DataFrame(columns=["Alias", "Text", "Web", "Export", "Data", "Type", "Att"])
        self.toolbar = Toolbar()
        self.table = Table(self.dataframe)
        self.properties = Properties()

        cell1 = new_cell(self.toolbar.content, height='50px')
        cell2 = new_cell(self.table.content, width='350px', height='250px', scroll=True)
        cell3 = new_cell(self.properties.content, width='600px', height='auto', scroll="Vertical")
        
        self.dev_n_qa = new_cell(HTML(), width='auto', height='650px')
        cell5 = new_cell(HTML(), width='auto', height='600px')
        footer = widgets.HBox([self.dev_n_qa, cell5])

        self.hbox = widgets.HBox([cell2, cell3], layout=widgets.Layout(height='300px'))
        self.content = widgets.VBox([cell1, self.hbox, footer], layout=widgets.Layout(height='auto'))

        _window = self
        
        # new window event
        def try_fire_new_window_event(self, b=None):
            if _window.new_window_event is not None:
                _window.new_window_event()
        _window.toolbar.new_window_button.on_click(try_fire_new_window_event)

        # inspect event
        def try_fire_inspect_event(self, b=None):
            if _window.inspect_event is not None:
                _window.inspect_event(_window.table, _window.properties)
        _window.toolbar.inspect_button.on_click(try_fire_inspect_event)    
        _window.table.data_event = _window.properties.reload


    def set_dev_n_qa(self, readme):
        self.dev_n_qa.children = [HTML(value=readme)]
        self.dev_n_qa.layout.display = 'none' if readme is None else 'block'


    def redraw(self):
        clear_output(wait=True)
        display(self.content)


    def reload(self, df=None, props={}):
        if df is None: return     
        self.dataframe = df
        self.table.reload(df)        

### Events

In [177]:
def new_window_clicked():

    global test_window
    try: test_window.quit()
    except: pass
    test_window = new_window("https://life.stg.wellzesta.com/login")


def on_inspect_clicked(table=None, properties=None): 

    global test_window
    if test_window is None: return

    metadata = read_page_objects_metadata(test_window)
    rows = []

    for tag_name, elements in metadata.items():
        for element in elements:

            att = element.pop('attributes', {})
            clazz =  att.pop('class', [])

            for k, v in att.items():
                element[k] = v

            element['class'] = clazz
            rows.append({
                "Export": False,
                "Alias": element.get('key', ''),
                "Web": element.get('selector', ''),
                "Type": tag_name,
                "Data": element,
            })

    df = pd.DataFrame(rows)
    table.reload(df)
    properties.reload({})

README = """<i>
When a DEV loves a QA <br/>
Can't keep his mind on any code <br/>
He'd debug all the errors <br/>
For the quality she's known <br/>
 <br/>
Yes, she's tough, oh so demanding <br/>
But he's the champ, codes so well <br/>
Even writes a testing framework <br/>
Just to make her world excel <br/>
 <br/>
Turns his back on his pattern <br/>
If they don't pass her test <br/>
When a DEV loves a QA <br/>
He's precise in every line <br/>
Crafting codes she envisioned <br/>
 <br/>
He'd give up all his code-tricks <br/>
And embrace the cleaner code <br/>
If she said that's the way <br/>
It ought to be <br/>
 <br/>
Though she's strict, he's so adept <br/>
Together they're a perfect duet
</i>"""



## Playground

In [241]:
def start_playground():
    window = Window(new_window_clicked, on_inspect_clicked)
    window.set_dev_n_qa(README)
    window.redraw()
    
start_playground() 